# Parsing a Counter-Strike 2 Demo

In this notebook, we show how to install Awpy and parse a Counter-Strike 2 demo file (also called a replay file). To start, install Awpy via pip by running `pip install awpy`. Python >= 3.9 is a prerequisite! You can get a demo either through the game itself, or by visiting a website like [HLTV](https://www.hltv.org/) or [FACEIT](https://www.faceit.com/). Then, to parse the demo, you can run the following:

In [1]:
from awpy import Demo

# Demo: https://www.hltv.org/matches/2372746/spirit-vs-natus-vincere-blast-premier-spring-final-2024 (de_dust2, Map 2)
dem = Demo("spirit-vs-natus-vincere-m2-dust2.dem", verbose=False)

# Available properties (all demos)
print(f"Kills: \n{dem.kills.head(n=3)}")
print(f"\nDamages: \n{dem.damages.head(n=3)}")
print(f"\nBomb: \n{dem.bomb.head(n=3)}")
print(f"\nSmokes: \n{dem.smokes.head(n=3)}")
print(f"\nInfernos: \n{dem.infernos.head(n=3)}")
print(f"\nWeapon Fires: \n{dem.weapon_fires.head(n=3)}")
print(f"\nRounds: \n{dem.rounds.head(n=3)}")
print(f"\nGrenades: \n{dem.grenades.head(n=3)}")
print(f"\nTicks: \n{dem.ticks.head(n=3)}")

print(f"\nHeader: \n{dem.header}")

Kills: 
    tick  assistedflash  dmg_health  dmg_armor  attackerblind  headshot  \
2   8359          False         110          0           True      True   
3  10302          False          10          5          False     False   
4  10989          False         117          0          False      True   

  hitgroup  noscope  penetrated  thrusmoke  ...  victim_current_equip_value  \
2     head    False           0      False  ...                         850   
3    chest    False           0      False  ...                        1250   
4     head    False           0      False  ...                         950   

  victim_has_defuser  victim_has_helmet                  victim_inventory  \
2              False              False        [knife_karambit, Glock-18]   
3              False              False  [knife_butterfly, Dual Berettas]   
4              False              False   [knife_karambit, Dual Berettas]   

   victim_ping  victim_team_name  victim_team_clan_name victim_na

### Changing parsing options
You can control the parser through various parameters, such as setting `ticks = False` if you do not want to parse ticks or `rounds=False` if you do not want to parse round information. Note that the round information is needed to produce the "top-level" dataframes, like `dem.kills`, `dem.damages` and so on. So, we recommend setting `rounds` to `False` only for debugging purposes or if you have your own flow.

In [2]:
import time

# With ticks
start = time.time()
dem_noticks = Demo("spirit-vs-natus-vincere-m2-dust2.dem", verbose=False, ticks=False)
end = time.time()

print(f"Parse time without parsing ticks: {end - start}")

Parse time without parsing ticks: 4.98794960975647


In [3]:
# Without tick data (much faster)
start = time.time()
dem_ticks = Demo("spirit-vs-natus-vincere-m2-dust2.dem", verbose=True, ticks=True)
end = time.time()

print(f"Parse time with parsing ticks: {end - start}")

2024-06-21 15:35:11.271 | SUCCESS  | awpy.demo:_success:150 - Created parser for spirit-vs-natus-vincere-m2-dust2.dem
2024-06-21 15:35:11.887 | DEBUG    | awpy.demo:_debug:168 - Found the following game events: ['round_freeze_end', 'player_given_c4', 'player_spawn', 'cs_pre_restart', 'player_sound', 'round_announce_match_start', 'player_team', 'cs_win_panel_match', 'flashbang_detonate', 'begin_new_match', 'round_time_warning', 'player_death', 'player_hurt', 'player_ping', 'round_announce_match_point', 'weapon_fire', 'announce_phase_end', 'hltv_chase', 'bomb_dropped', 'bomb_pickup', 'bomb_defused', 'smokegrenade_expired', 'switch_team', 'item_pickup', 'cs_intermission', 'cs_round_start_beep', 'bomb_exploded', 'hltv_fixed', 'entity_killed', 'vote_cast', 'server_cvar', 'smokegrenade_detonate', 'round_officially_ended', 'inferno_startburn', 'round_end_upload_stats', 'bomb_planted', 'player_ping_stop', 'hltv_versioninfo', 'cs_round_final_beep', 'hegrenade_detonate', 'inferno_expire', 'round

Parse time with parsing ticks: 18.557005167007446


### Getting player and global properties
Awpy uses [demoparser2](https://github.com/LaihoE/demoparser) as its parsing backend. This means that you can pass a list of `player_props` or `other_props`. In the following example, we get some player position properties. If you do not pass any props, we choose a default list of properties (which is already quite extensive). To see a list of available properties, visit demoparser2's repository.

In [4]:
dem_with_props = Demo("spirit-vs-natus-vincere-m2-dust2.dem", player_props=["X", "Y"])

dem_with_props.ticks[["tick", "team_name", "name", "X", "Y", "inventory"]].sample(n=10)

,tick,team_name,name,X,Y,inventory
165399,15561,TERRORIST,b1t,747.968750,235.968750,"[knife_karambit, Glock-18]"
383060,35926,CT,donk,1492.246094,2666.138916,"[knife_butterfly, AK-47, Desert Eagle]"
1903064,178457,CT,iM,916.031250,1195.968750,"[knife_karambit, USP-S, AK-47, High Explosive ..."
497887,46678,TERRORIST,jL,-2151.559814,1846.127319,[]
2154235,201787,CT,Aleksib,649.847473,2258.413086,"[knife_butterfly, USP-S, M4A1-S, Smoke Grenade..."
1844316,172999,TERRORIST,chopper,998.816650,556.601929,"[knife_stiletto, Glock-18, Galil AR, C4]"
1864509,174992,CT,b1t,1167.529297,2814.498535,"[knife_butterfly, USP-S, AWP]"
1541011,144569,TERRORIST,magixx,391.178009,-318.466187,"[knife_butterfly, Glock-18, Galil AR, Smoke Gr..."
1739396,163150,TERRORIST,chopper,-873.285583,1438.879395,[]
1461279,137371,CT,b1t,1263.826782,2926.308594,[]


### Obtaining all events
Because we use the demoparser2 backend, we store all events in the demo. You can access these events with the `.events` property, which is a dictionary.

In [5]:
dem = Demo("spirit-vs-natus-vincere-m2-dust2.dem", ticks=False)

for event_name, event in dem.events.items():
    print(f"{event_name}: {event.shape[0]} rows x {event.shape[1]} columns")

round_freeze_end: 23 rows x 16 columns
round_announce_match_start: 1 rows x 16 columns
round_announce_last_round_half: 1 rows x 16 columns
player_given_c4: 24 rows x 36 columns
inferno_expire: 108 rows x 40 columns
player_hurt: 588 rows x 62 columns
cs_round_start_beep: 69 rows x 16 columns
round_announce_match_point: 1 rows x 16 columns
bomb_exploded: 6 rows x 38 columns
inferno_startburn: 108 rows x 40 columns
switch_team: 20 rows x 21 columns
player_ping: 75 rows x 41 columns
hltv_chase: 453 rows x 23 columns
cs_intermission: 1 rows x 16 columns
smokegrenade_detonate: 146 rows x 40 columns
hltv_fixed: 132 rows x 24 columns
round_end_upload_stats: 23 rows x 16 columns
bomb_planted: 12 rows x 38 columns
player_sound: 23783 rows x 39 columns
player_spawn: 242 rows x 36 columns
player_death: 165 rows x 95 columns
entity_killed: 165 rows x 20 columns
hltv_versioninfo: 1 rows x 17 columns
cs_pre_restart: 23 rows x 16 columns
bomb_pickup: 75 rows x 16 columns
player_ping_stop: 45 rows x 37

### Printing logs
The `Demo` class can output helpful logs if you enable `verbose=True`.

In [6]:
dem = Demo("spirit-vs-natus-vincere-m2-dust2.dem", verbose=True)

2024-06-21 15:35:53.381 | SUCCESS  | awpy.demo:_success:150 - Created parser for spirit-vs-natus-vincere-m2-dust2.dem
2024-06-21 15:35:54.001 | DEBUG    | awpy.demo:_debug:168 - Found the following game events: ['vote_cast', 'bomb_pickup', 'player_given_c4', 'hegrenade_detonate', 'smokegrenade_expired', 'switch_team', 'round_officially_ended', 'cs_round_final_beep', 'player_team', 'bomb_dropped', 'server_cvar', 'round_freeze_end', 'smokegrenade_detonate', 'bomb_planted', 'round_time_warning', 'cs_win_panel_match', 'inferno_expire', 'bomb_defused', 'player_death', 'begin_new_match', 'round_announce_match_start', 'entity_killed', 'hltv_chase', 'cs_round_start_beep', 'item_pickup', 'hltv_versioninfo', 'player_ping_stop', 'round_end_upload_stats', 'hltv_fixed', 'cs_pre_restart', 'bomb_exploded', 'weapon_fire', 'player_sound', 'player_ping', 'player_spawn', 'cs_intermission', 'flashbang_detonate', 'inferno_startburn', 'announce_phase_end', 'round_announce_match_point', 'player_hurt', 'round